Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Nifti Import

**From Directory**
___

Here's a brief markup (in Markdown format) that explains the purpose and usage of the `segments_dict`:

---

## Neuroimaging File Extraction Dictionary

The `segments_dict` is a predefined dictionary structured to facilitate the extraction of specific types of neuroimaging files. Each key in the dictionary represents a distinct neuroimaging segment, and its associated value is another dictionary containing the following fields:

- **path**: This should be filled with the absolute path to the base directory containing the neuroimaging files for the corresponding segment. 
- **glob_name_pattern**: This is the string pattern that will be used to "glob" or search for the specific files within the provided path. It helps in identifying and extracting the desired files based on their naming conventions.

Here's a breakdown of the segments and their respective fields:

### 1. Cerebrospinal Fluid (CSF)
- **path**: Absolute path to the base directory containing CSF files.
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **path**: Absolute path to the base directory containing grey matter files.
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **path**: Absolute path to the base directory containing white matter files.
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `path` and `glob_name_pattern` fields for each segment in the `segments_dict`. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.
- < *_name_pattern > variables do not need a leading slash ("/"). This is already accounted for. 

---

# Import Segmented Patients for Atrophy Detection

In [2]:
base_directory = r'/Volumes/Expansion/datasets/adni/neuroimaging/all_patients_by_diagnosis_symlink/Additional'
grey_matter_glob_name_pattern = '*/mri/*mwp1*resampled*'
white_matter_glob_name_pattern = '*/mri/*mwp2*resampled*'
csf_glob_name_pattern = '*/mri/*mwp3*resampled*'

In [ ]:
from calvin_utils.vbm_utils.loading import import_dataframes_from_folders
dataframes_dict = import_dataframes_from_folders(base_directory, grey_matter_glob_name_pattern, white_matter_glob_name_pattern, csf_glob_name_pattern)

**Extract Subject ID From File Names**
- Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. If nothing preceding subject identifier, enter "".
- Do NOT include mwp[1/2/3] in this. 
2) Proceeding the subject ID. If nothing proceeding subject identifier, enter "".

In [4]:
preceding_id = ''
proceeding_id = '_resam'

In [ ]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects

dataframes_dict = rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id)

# Import Control Segments

In [6]:
base_directory_control = '/Volumes/Expansion/datasets/adni/neuroimaging/true_control/anat/mri'
control_grey_matter_glob_name_pattern = '*smwp1*resampled*'
control_white_matter_glob_name_pattern = '*smwp2*resampled*'
control_csf_glob_name_pattern = '*smwp3*resampled*'

In [ ]:
from calvin_utils.vbm_utils.loading import import_control_dataframes
control_dataframes_dict = import_control_dataframes(base_directory_control, control_grey_matter_glob_name_pattern, control_white_matter_glob_name_pattern, control_csf_glob_name_pattern)

**Extract Subject ID From File Names**
- Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. If nothing preceding subject identifier, enter "".
- **Do NOT include mwp[1/2/3] in this.**
2) Proceeding the subject ID. If nothing proceeding subject identifier, enter "".


- The example filenames were all provided above

In [8]:
preceding_id = 's'
proceeding_id = '_re'

In [ ]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects

control_dataframes_dict = rename_dataframe_subjects(control_dataframes_dict, preceding_id, proceeding_id)

# Import Covariates
Expects a CSV as below: 
```
+---------+----------------------------+--------------+--------------+--------------+
|Covariate| Subject 1                  | Subject 2    | Subject . . .| Subject N    |
+---------+----------------------------+--------------+--------------+--------------+
| Male    | 0                          | 1            | 1            | 1            |
| Female  | 1                          | 0            | 0            | 0            |
| Age     | 65                         | 72           | 87           | 90           |
+---------+----------------------------+--------------+--------------+--------------+
```
**1 is True, 0 is False, Age is represented in years.**

In [10]:
control_covariates_csv_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/metadata/paths_and_covariates/control_group_covariates.csv'
patient_covariates_csv_path = '/Volumes/Expansion/datasets/adni/metadata/cleaned_information/wmap_demographics.csv'

In [ ]:
from calvin_utils.vbm_utils.w_scoring import import_covariates
control_covariates_df, patient_covariates_df = import_covariates(control_covariates_csv_path=control_covariates_csv_path, patient_covariates_csv_path=patient_covariates_csv_path)
control_covariates_df

In [ ]:
patient_covariates_df

Improve the Naming of the Covariate Subjects
- Pending construction. Should have named it appropraitely when you built your CSV. 
- Will code PRN. 

# Generate W-Scored Atrophy Maps for Each Segment

In [ ]:
from calvin_utils.vbm_utils.w_scoring  import CalvinWMap
wmapper = CalvinWMap(dataframes_dict=dataframes_dict, control_dataframes_dict=control_dataframes_dict, control_covariates_df=control_covariates_df, patient_covariates_df=patient_covariates_df)
atrophy_dataframes_dict, significant_atrophy_dataframes_dict = wmapper.run()

**Generate Composite Maps**

In [15]:
from calvin_utils.vbm_utils.composite_atrophy_mapper import generate_composite_maps
unthresholded_atrophy_dataframes_dict = generate_composite_maps(atrophy_dataframes_dict, threshold=False)
thresholded_atrophy_dataframes_dict = generate_composite_maps(significant_atrophy_dataframes_dict, threshold=True)

# Save the W-Scored Maps

Unthresholded Maps

In [ ]:
base_directory='/Volumes/Expansion/datasets/adni/neuroimaging/all_patients_atrophy_seeds'
session = 'pre-tms'

Must provide a mask path. One is available in /assets

In [ ]:
mask_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/assets/MNI152_T1_2mm_brain.nii'

In [ ]:
from calvin_utils.vbm_utils.processing import save_nifti_to_bids
save_nifti_to_bids(unthresholded_atrophy_dataframes_dict, 
                   bids_base_dir=base_directory, 
                   mask_path=mask_path,
                   analysis='tissue_segment_w_scores', 
                   ses=session,
                   dry_run=False);

Thresholded Maps - The 'Real' Atrophy


In [ ]:
from calvin_utils.vbm_utils.processing import save_nifti_to_bids
save_nifti_to_bids(unthresholded_atrophy_dataframes_dict, 
                   bids_base_dir=base_directory, 
                   mask_path=mask_path,
                   analysis='thresholded_tissue_segment_w_scores', 
                   ses=session,
                   dry_run=False);

In [ ]:
import os
os.system('say "Atrophy seeds are complete."')

All Done. Enjoy your atrophy seeds.

--Calvin